In [1]:
import os
import sys
sys.path.append("/home/xucong24/Compiler")
from src.rl.llvm_wrapper import llvm_wrapper, PassformerObservation
from src.model import PassformerModel, PassformerConfig, Inst2VecTokenizer, OptiSeqTokenizer
from datasets import Dataset
import gym
import compiler_gym

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import copy

In [2]:
from compiler_gym.envs.llvm import make_benchmark

model_id = "/home/xucong24/Compiler/work_dirs/passformer_gallvm_seq2seq_concat/20260120_195517/final_model"
encoder_tokenizer_id = "/home/xucong24/Compiler/checkpoints/Inst2VecTokenizer"
decoder_tokenizer_id = "/home/xucong24/Compiler/checkpoints/OptiSeqTokenizer"

# model = PassformerModel.from_pretrained(model_id)
encoder_tokenizer = Inst2VecTokenizer.from_pretrained(encoder_tokenizer_id)
decoder_tokenizer = OptiSeqTokenizer.from_pretrained(decoder_tokenizer_id)


bc_files = [
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_dijkstra.bc", 
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_crc32.bc",
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_dijkstra.bc", 
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_crc32.bc",
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_dijkstra.bc", 
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_crc32.bc",
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_dijkstra.bc", 
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_crc32.bc",
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_dijkstra.bc", 
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_crc32.bc"
    ]

def prepare_dataset(bc_files, encoder_tokenizer):
    all_data = {
        "llvm_ir": [],
        "autophase": [],    # 原始 autophase 特征
        "bc_path": []       # 用于 Reward 计算
    }
    
    for bc in bc_files:
        # 1. 环境初始化
        env = llvm_wrapper([bc], is_from_bc=True)
        observation = env.reset()
        all_data["llvm_ir"].append(observation.llvm_ir)
        all_data["autophase"].append(observation.autophase)
        all_data["bc_path"].append(bc)
        env.close()
        
    return Dataset.from_dict(all_data)

datasets = prepare_dataset(bc_files, encoder_tokenizer,)
print(datasets)


/home/xucong24/Compiler/.venv/lib/python3.10/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


Dataset({
    features: ['llvm_ir', 'autophase', 'bc_path'],
    num_rows: 10
})


In [3]:
datasets[0]

{'llvm_ir': '; ModuleID = \'-\'\nsource_filename = "-"\ntarget datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"\ntarget triple = "x86_64-unknown-linux-gnu"\n\n%struct._QITEM = type { i32, i32, i32, %struct._QITEM* }\n%struct._IO_FILE = type { i32, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, %struct._IO_marker*, %struct._IO_FILE*, i32, i32, i64, i16, i8, [1 x i8], i8*, i64, %struct._IO_codecvt*, %struct._IO_wide_data*, %struct._IO_FILE*, i8*, i64, i32, [20 x i8] }\n%struct._IO_marker = type opaque\n%struct._IO_codecvt = type opaque\n%struct._IO_wide_data = type opaque\n%struct._NODE = type { i32, i32 }\n\n@NUM_NODES = dso_local global i32 0, align 4\n@qHead = dso_local global %struct._QITEM* null, align 8\n@g_qCount = dso_local global i32 0, align 4\n@.str = private unnamed_addr constant [4 x i8] c" %d\\00", align 1\n@stdout = external dso_local global %struct._IO_FILE*, align 8\n@stderr = external dso_local global %struct._IO_FILE*, align

In [6]:
def collate_fn(batch):
    return {
        'llvm_ir': [x['llvm_ir'] for x in batch],
        'autophase': torch.stack([torch.tensor(x['autophase']) for x in batch]),
        'bc_path': [x['bc_path'] for x in batch]
    }

loader = DataLoader(datasets, batch_size=2, shuffle=True, collate_fn=collate_fn)
for batch in loader:
    print(batch)

{'llvm_ir': ['; ModuleID = \'-\'\nsource_filename = "-"\ntarget datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"\ntarget triple = "x86_64-unknown-linux-gnu"\n\n%struct._IO_FILE = type { i32, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, %struct._IO_marker*, %struct._IO_FILE*, i32, i32, i64, i16, i8, [1 x i8], i8*, i64, %struct._IO_codecvt*, %struct._IO_wide_data*, %struct._IO_FILE*, i8*, i64, i32, [20 x i8] }\n%struct._IO_marker = type opaque\n%struct._IO_codecvt = type opaque\n%struct._IO_wide_data = type opaque\n\n@crc_32_tab = internal global [256 x i64] [i64 0, i64 1996959894, i64 3993919788, i64 2567524794, i64 124634137, i64 1886057615, i64 3915621685, i64 2657392035, i64 249268274, i64 2044508324, i64 3772115230, i64 2547177864, i64 162941995, i64 2125561021, i64 3887607047, i64 2428444049, i64 498536548, i64 1789927666, i64 4089016648, i64 2227061214, i64 450548861, i64 1843258603, i64 4107580753, i64 2211677639, i64 325883990, i64 

In [7]:
model = PassformerModel.from_pretrained(model_id)
model

PassformerModel(
  (encoder): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(8569, 768, padding_idx=8565)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, bias=False)
        )
      )
      (1-21): 21 x

In [8]:
def compiler_gym_reward_fn(completions, bc_path, **kwargs):
    """
    completions: 模型生成的 Pass 字符串列表 (e.g., ["-mem2reg -gvn", "-O3"])
    bc_path: 对应的位码文件路径
    """
    rewards = []
    print(completions)
    
    # 注意：GRPO 会批量传入数据
    for completion, path in zip(completions, bc_path):
        try:
            # 1. 创建临时的 CompilerGym 环境
            # 或者使用全局环境重置：env.reset(path)
            temp_env = llvm_wrapper([path], is_from_bc=True)
            observation = temp_env.reset()
            
            # 2. 执行生成的 Pass 序列
            # 假设你的 completion 是 "pass1 pass2"
            _, reward, done, info = temp_env.multistep_by_action_flags(completion)
            
            # 3. 记录奖励（例如使用代码体积变化率）
            rewards.append(float(reward))
            
            temp_env.close()
        except Exception as e:
            print(f"Error processing {path}: {e}")
            rewards.append(-1.0) # 编译失败或崩溃给予惩罚
            
    return rewards

completions = ["-mem2reg -gvn" for i in range(10)]
compiler_gym_reward_fn(completions, bc_files)

['-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn']


/home/xucong24/Compiler/.venv/lib/python3.10/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


[0.8469387755102041,
 0.9765625,
 0.8469387755102041,
 0.9765625,
 0.8469387755102041,
 0.9765625,
 0.8469387755102041,
 0.9765625,
 0.8469387755102041,
 0.9765625]

In [ ]:
class GRPOTrainerManual:
    def __init__(self, model, encoder_tokenizer, decoder_tokenizer, reward_fn, 
                 beta=0.01, eps=0.2, lr=1e-5, num_generations=8):
        self.model = model
        # 参考模型，用于计算 KL 散度，保持冻结
        self.ref_model = copy.deepcopy(model).eval()
        self.encoder_tokenizer = encoder_tokenizer
        self.decoder_tokenizer = decoder_tokenizer
        self.reward_fn = reward_fn
        
        self.beta = beta           # KL 惩罚系数
        self.eps = eps             # PPO 剪切阈值
        self.num_gens = num_generations
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    def get_log_probs(self, logits, tokens):
        """计算生成的 token 的 log 概率"""
        log_probs = F.log_softmax(logits, dim=-1)
        # 选取实际生成的 token 对应的概率
        per_token_logps = torch.gather(log_probs, dim=2, index=tokens.unsqueeze(-1)).squeeze(-1)
        return per_token_logps

    def train_step(self, batch):
        """
        batch: 包含 'llvm_ir' (IR文本), 'autophase', 'bc_path'
        """
        device = self.model.device
        llvm_irs = batch['llvm_ir']
        autophases = batch['autophase'].to(device)
        bc_paths = batch['bc_path']

        # --- 1. 采样阶段 (Rollout) ---
        self.model.eval()
        with torch.no_grad():
            # 为每个 llvm_ir 生成多组序列
            # 编码输入
            inputs = self.encoder_tokenizer(llvm_irs, padding=True, max_length=512, return_tensors="pt")
            
            # 生成结果
            # 注意：这里需要重复输入以实现 Batch 生成
            expanded_input_ids = inputs['input_ids'].repeat_interleave(self.num_gens, dim=0).to(device)
            expanded_attention_mask = inputs['attention_mask'].repeat_interleave(self.num_gens, dim=0).to(device)
            expanded_autophase = autophases.repeat_interleave(self.num_gens, dim=0).to(device)
            
            output_ids = self.model.generate(
                input_ids=expanded_input_ids,
                attention_mask=expanded_attention_mask,
                autophase=expanded_autophase,
                max_length=20,
                do_sample=True # 必须开启采样以保证多样性
            )
            # 提取生成的部分 (假设 output_ids 包含 llvm_ir，需切片；若模型直接返回 completion 则不用)
            completions = output_ids 
            
            # 解码为文本以获取 Reward
            completion_texts = self.decoder_tokenizer.batch_decode(completions, skip_special_tokens=True)

        # --- 2. 奖励计算 (Reward) ---
        # 扩展 bc_paths 以匹配生成数量
        expanded_bc_paths = [p for p in bc_paths for _ in range(self.num_gens)]
        rewards = self.reward_fn(completion_texts, expanded_bc_paths)
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)

        # --- 3. 计算优势 (Advantage) ---
        # 按组归一化：(r - mean) / std
        rewards_reshaped = rewards.view(-1, self.num_gens)
        mean_r = rewards_reshaped.mean(dim=1, keepdim=True)
        std_r = rewards_reshaped.std(dim=1, keepdim=True) + 1e-8
        advantages = ((rewards_reshaped - mean_r) / std_r).view(-1)

        # --- 4. 优化阶段 (Update) ---
        self.model.train()
        # 计算当前模型的 log_probs
        outputs = self.model(
            input_ids=expanded_input_ids,
            attention_mask=expanded_attention_mask,
            autophase=expanded_autophase,
            # labels=completions # 强制模型计算这些 token 的 logits
            decoder_input_ids=completions,          # 明确指定 Decoder 输入
            labels=completions                      # 用于计算 logits 和 loss
        )
        current_log_probs = self.get_log_probs(outputs.logits, completions)
        
        # 计算参考模型的 log_probs (用于 KL)
        with torch.no_grad():
            ref_outputs = self.ref_model(
                input_ids=expanded_input_ids,
                attention_mask=expanded_attention_mask,
                autophase=expanded_autophase,
                decoder_input_ids=completions,          # 明确指定 Decoder 输入
                labels=completions                      # 用于计算 logits 和 loss
            )
            ref_log_probs = self.get_log_probs(ref_outputs.logits, completions)

        # 重要性采样 Ratio (这里简化处理，假设采样时的 log_probs 与 current 近似)
        # 在标准的 GRPO 中，Ratio 是 current / old_sampling
        ratio = torch.exp(current_log_probs.sum(dim=-1) - current_log_probs.detach().sum(dim=-1)) 
        
        # PPO Clip Loss
        surr1 = ratio * advantages
        surr2 = torch.clamp(ratio, 1 - self.eps, 1 + self.eps) * advantages
        policy_loss = -torch.min(surr1, surr2).mean()

        # KL Loss (避免偏离参考模型太远)
        kl = torch.exp(ref_log_probs - current_log_probs) - (ref_log_probs - current_log_probs) - 1
        kl_loss = self.beta * kl.mean()

        total_loss = policy_loss + kl_loss

        self.optimizer.zero_grad()
        total_loss.backward()
        self.optimizer.step()

        return total_loss.item(), rewards.mean().item()

In [18]:
my_trainer = GRPOTrainerManual(
    model=model,
    encoder_tokenizer=encoder_tokenizer,
    decoder_tokenizer=decoder_tokenizer,
    reward_fn=compiler_gym_reward_fn,
    num_generations=8
)

# 3. 训练循环
for epoch in range(3):
    pbar = tqdm(loader)
    for batch in pbar:
        loss, avg_reward = my_trainer.train_step(batch)
    pbar.set_description(f"Loss: {loss:.4f} | Reward: {avg_reward:.4f}")

  0%|          | 0/5 [00:00<?, ?it/s]

['-loop-interchange -sink -reg2mem -insert-gcov-profiling -separate-const-offset-from-gep -mergereturn -lcssa -loop-rotate -barrier -infer-address-spaces -licm -lower-widenable-condition -loop-simplify -tailcallelim -loop-unroll-and-jam -partial-inliner -dse -strip-debug-declare', '-lower-expect -loop-rotate -instnamer -insert-gcov-profiling -separate-const-offset-from-gep -lower-expect -lcssa -forceattrs -barrier -loop-reduce -strip-nondebug -lower-widenable-condition -licm -tailcallelim -loweratomic -loop-rotate -dce -strip-debug-declare', '-lower-expect -loop-rotate -instnamer -insert-gcov-profiling -separate-const-offset-from-gep -lower-expect -mem2reg -forceattrs -barrier -loop-reduce -strip-nondebug -lower-widenable-condition -licm -tailcallelim -loweratomic -loop-rotate -dce -strip-debug-declare', '-coro-early -loop-simplify -loop-load-elim -ipconstprop -lower-matrix-intrinsics -mergeicmps -constmerge -sancov -barrier -guard-widening -barrier -lower-widenable-condition -lcssa -r

/home/xucong24/Compiler/.venv/lib/python3.10/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(
  0%|          | 0/5 [00:42<?, ?it/s]


ValueError: You have to specify either input_ids or inputs_embeds

In [ ]:
['-loop-interchange -sink -reg2mem -insert-gcov-profiling -separate-const-offset-from-gep -mergereturn -lcssa -loop-rotate -barrier -infer-address-spaces -licm -lower-widenable-condition -loop-simplify -tailcallelim -loop-unroll-and-jam -partial-inliner -dse -strip-debug-declare', 
'-lower-expect -loop-rotate -instnamer -insert-gcov-profiling -separate-const-offset-from-gep -lower-expect -lcssa -forceattrs -barrier -loop-reduce -strip-nondebug -lower-widenable-condition -licm -tailcallelim -loweratomic -loop-rotate -dce -strip-debug-declare', '-lower-expect -loop-rotate -instnamer -insert-gcov-profiling -separate-const-offset-from-gep -lower-expect -mem2reg -forceattrs -barrier -loop-reduce -strip-nondebug -lower-widenable-condition -licm -tailcallelim -loweratomic -loop-rotate -dce -strip-debug-declare', '-coro-early -loop-simplify -loop-load-elim -ipconstprop -lower-matrix-intrinsics -mergeicmps -constmerge -sancov -barrier -guard-widening -barrier -lower-widenable-condition -lcssa -reassociate -loweratomic -rewrite-statepoints-for-gc -dce -strip-debug-declare', '-loop-interchange -sink -reg2mem -insert-gcov-profiling -separate-const-offset-from-gep -mergereturn -lcssa -loop-rotate -barrier -loop-reduce -licm -lower-widenable-condition -loop-simplify -tailcallelim -loop-unroll-and-jam -early-cse-memssa -dse -strip-debug-declare', '-ee-instrument -sink -reg2mem -insert-gcov-profiling -licm -mergereturn -lcssa -loop-rotate -barrier -infer-address-spaces -loop-unroll-and-jam -lower-widenable-condition -callsite-splitting -tailcallelim -loop-unroll-and-jam -loop-simplify -dse -strip-debug-declare', '-ee-instrument -instnamer -loop-instsimplify -name-anon-globals -partially-inline-libcalls -mergeicmps -load-store-vectorizer -early-cse-memssa -globalsplit -instsimplify -inferattrs -mergereturn -argpromotion -inline -mem2reg -loop-rotate -loop-distribute -loop-deletion', '-name-anon-globals -loop-vectorize -newgvn -deadargelim -lower-matrix-intrinsics -mergeicmps -nary-reassociate -loop-interchange -early-cse -loop-interchange -inferattrs -mergeicmps -attributor -called-value-propagation -loweratomic -loop-versioning-licm -speculative-execution -simple-loop-unswitch', '-irce -simple-loop-unswitch -slsr -rewrite-statepoints-for-gc -lower-constant-intrinsics -instsimplify -lower-matrix-intrinsics -loop-unroll-and-jam -loop-versioning-licm -functionattrs -constmerge -globalsplit -globalopt -loop-interchange -div-rem-pairs -globalsplit -strip-dead-prototypes -reg2mem', '-mldst-motion -loop-deletion -indvars -hotcoldsplit -dce -loop-load-elim -load-store-vectorizer -float2int -guard-widening -float2int -sancov -deadargelim -mergeicmps -irce -rpo-functionattrs -globalopt -libcalls-shrinkwrap -indvars', '-pgo-memop-opt -mergefunc -separate-const-offset-from-gep -lower-widenable-condition -loop-rotate -newgvn -loop-load-elim -callsite-splitting -load-store-vectorizer -dce -slp-vectorizer -coro-cleanup -float2int -alignment-from-assumptions -loop-reroll -elim-avail-extern -always-inline -instnamer', '-libcalls-shrinkwrap -loop-unroll -slsr -slp-vectorizer -lower-constant-intrinsics -instsimplify -lower-matrix-intrinsics -simplifycfg -loop-versioning-licm -tailcallelim -lowerswitch -indvars -deadargelim -loop-interchange -div-rem-pairs -loop-unroll -dce -loop-data-prefetch', '-loop-rotate -nary-reassociate -rpo-functionattrs -lcssa -ee-instrument -name-anon-globals -jump-threading -callsite-splitting -loop-unroll-and-jam -load-store-vectorizer -loop-idiom -globalsplit -div-rem-pairs -ee-instrument -loop-simplify -tailcallelim -pgo-memop-opt -gvn-hoist', '-die -lower-widenable-condition -irce -aggressive-instcombine -aggressive-instcombine -simple-loop-unswitch -bdce -mergereturn -loop-versioning-licm -consthoist -sancov -loop-rotate -name-anon-globals -barrier -loop-sink -argpromotion -coro-cleanup -slp-vectorizer', '-die -early-cse-memssa -slsr -memcpyopt -lower-constant-intrinsics -coro-cleanup -loop-load-elim -simplifycfg -deadargelim -mergefunc -loop-reroll -indvars -loop-reroll -barrier -mldst-motion -loop-data-prefetch -coro-cleanup -loop-unroll', '-mldst-motion -instnamer -coro-elide -lower-expect -adce -barrier -post-inline-ee-instrument -loop-interchange -prune-eh -licm -always-inline -ipsccp -loop-versioning -coro-early -lower-constant-intrinsics -loop-reroll -aggressive-instcombine -partially-inline-libcalls']